# DE32 Image Processing Notebook

See './Image_Analysis/Examples/1_Core_Image_Processing.ipynb' for a documented example of notebook usage.

#### Imports

In [ ]:
import paulssonlab.deaton.trenchripper.trenchripper as tr

import warnings

warnings.filterwarnings(action="once")

import matplotlib

matplotlib.rcParams["figure.figsize"] = [20, 10]

In [ ]:
# addition of active memory manager
import dask
dask.config.set({'distributed.scheduler.active-memory-manager.start': True});
dask.config.set({'distributed.scheduler.worker-ttl': "5m"});
dask.config.set({'distributed.scheduler.allowed-failures': 100});

dask_wd = "/home/de64/scratch/de64/dask"

In [ ]:
### Dark Image

flatfieldpath = "/home/de64/scratch/de64/sync_folder/2024-03-15_Flat_Fields_Iris_9_20x/20X_Dark.nd2"
outputpath = "/home/de64/scratch/de64/sync_folder/2024-03-15_Flat_Fields_Iris_9_20x/20X_Dark.tiff"
tr.generate_flatfield(flatfieldpath,outputpath)

In [ ]:
### MCherry

flatfieldpath = "/home/de64/scratch/de64/sync_folder/2024-03-15_Flat_Fields_Iris_9_20x/20X_mCherry.nd2"
outputpath = "/home/de64/scratch/de64/sync_folder/2024-03-15_Flat_Fields_Iris_9_20x/20X_mCherry.tiff"
tr.generate_flatfield(flatfieldpath,outputpath)

In [ ]:
import tifffile
from matplotlib import pyplot as plt

In [ ]:
dark_outputpath = "/home/de64/scratch/de64/sync_folder/2024-03-15_Flat_Fields_Iris_9_20x/20X_Dark.tiff"
Mcherry_outputpath = "/home/de64/scratch/de64/sync_folder/2024-03-15_Flat_Fields_Iris_9_20x/20X_mCherry.tiff"

fflist = [dark_outputpath,Mcherry_outputpath]

fig, axs = plt.subplots(figsize=(5, 6), nrows= 1, ncols = 2)

for i in range(2):
    axs[i].imshow(tifffile.imread(fflist[i]))
    axs[i].set_title(fflist[i].split('/')[-1].split(".")[0])
plt.tight_layout()

# Part 1: Growth/Division

#### Specify Paths

In [ ]:
headpath = "/home/de64/scratch/de64/sync_folder/2024-03-06_DE32_32hours"
nd2file = "/home/de64/scratch/de64/sync_folder/2024-03-06_DE32_32hours/Experiment.nd2"

## Extract to hdf5 files

#### Start Dask Workers

In [ ]:
dask_controller = tr.trcluster.dask_controller(
    walltime="1:00:00",
    local=False,
    n_workers=200,
    n_workers_min=50,
    memory="8GB",
    working_directory=dask_wd,
)
dask_controller.startdask()

In [ ]:
dask_controller.displaydashboard()

##### Perform Extraction

In [ ]:
## note that the override is used to deal with corrupted metadata
hdf5_extractor = tr.hdf5_fov_extractor(
    nd2file,
    headpath,
    tpts_per_file=50,
    ignore_fovmetadata=False,
    nd2reader_override={"z_levels": [], "z_coordinates": [], "fields_of_view": list(range(629)) , "frames":list(range(165)), "num_fovs": 629, "num_frames": 165},
)

##### Extraction Parameters

In [ ]:
hdf5_extractor.inter_set_params()

#### List of paths to flat fields
 - /home/de64/scratch/de64/sync_folder/2024-03-15_Flat_Fields_Iris_9_20x/20X_mCherry.tiff
 - /home/de64/scratch/de64/sync_folder/2024-03-15_Flat_Fields_Iris_9_20x/20X_Dark.tiff

In [ ]:
hdf5_extractor.inter_set_flatfieldpaths()

##### Begin Extraction 

In [ ]:
hdf5_extractor.extract(dask_controller)

## Kymographs

### Test Parameters



##### Initialize the interactive kymograph class

In [ ]:
interactive_kymograph = tr.kymograph_interactive(headpath)

In [ ]:
interactive_kymograph.import_hdf5_interactive()

##### Tune "trench-row" detection hyperparameters

In [ ]:
interactive_kymograph.preview_y_precentiles_interactive()

##### Tune "trench-row" cropping hyperparameters

In [ ]:
interactive_kymograph.preview_y_precentiles_consensus_interactive()

In [ ]:
interactive_kymograph.preview_y_crop_interactive()

##### Tune trench detection hyperparameters

In [ ]:
interactive_kymograph.preview_x_percentiles_interactive()

##### Tune trench cropping hyperparameters

In [ ]:
interactive_kymograph.preview_kymographs_interactive()

##### Export and save hyperparameters

In [ ]:
interactive_kymograph.process_results()

In [ ]:
interactive_kymograph.write_param_file()

### Generate Kymograph

##### Perform Kymograph Cropping

In [ ]:
kymoclust = tr.kymograph.kymograph_cluster(
    headpath=headpath, paramfile=True
)

##### Begin Kymograph Cropping 

In [ ]:
kymoclust.generate_kymographs(dask_controller)

In [ ]:
lane_overlay_handle = tr.lane_overlay(headpath,persist_data=True)
overlay = lane_overlay_handle.view_overlay(size=2500,vmin=0,vmax=4000)

In [ ]:
overlay

In [ ]:
lane_overlay_handle.save_rows()

##### Post-process Images

In [ ]:
kymoclust.post_process(dask_controller,trench_timepoints_per_file=10000)

##### Check kymograph statistics

In [ ]:
kymoclust.kymo_report()

## Fluorescence Segmentation

### Test Parameters

#### Typical parameters for mCherry single channel imaging at 20x

##### Initialize the interactive segmentation class

In [ ]:
interactive_segmentation = tr.fluo_segmentation_interactive(headpath)

##### Choose channel to segment on

In [ ]:
interactive_segmentation.choose_seg_channel_inter()

#### Import data

In [ ]:
interactive_segmentation.import_array_inter()

##### Process data

In [ ]:
interactive_segmentation.plot_processed_inter()

#### Determine Cell Mask Envelope

In [ ]:
interactive_segmentation.plot_cell_mask_inter()

In [ ]:
interactive_segmentation.plot_eig_mask_inter()

In [ ]:
interactive_segmentation.plot_dist_mask_inter()

In [ ]:
interactive_segmentation.plot_marker_mask_inter()

In [ ]:
interactive_segmentation.process_results()

In [ ]:
interactive_segmentation.write_param_file()

#### Shutdown Dask

In [ ]:
dask_controller.shutdown()

### Generate Segmentation

#### Start Dask Workers

In [ ]:
dask_controller = tr.trcluster.dask_controller(
    walltime="1:00:00",
    local=False,
    n_workers=400,
    n_workers_min=20,
    memory="2GB",
    working_directory=dask_wd,
)
dask_controller.startdask()

In [ ]:
dask_controller.daskclient

In [ ]:
segment = tr.segment.fluo_segmentation_cluster(headpath, paramfile=True)

In [ ]:
segment.dask_segment(dask_controller)

#### Stop Dask Workers

In [ ]:
dask_controller.shutdown()

## Lineage Tracing

### Test Parameters

In [ ]:
score_function = tr.tracking.scorefn(
    headpath,
    "fluorsegmentation",
    size_attr='axis_major_length',
    u_size=0.08,
    sig_size=0.03,
    u_pos=0.1,
    sig_pos=0.05,
    w_merge=0.,
)

In [ ]:
score_function.interactive_scorefn()

In [ ]:
Tracking_Solver = tr.tracking.tracking_solver(
    headpath, "fluorsegmentation", ScoreFn=score_function, edge_limit=2,
)
data, orientation = score_function.output.result

In [ ]:
Tracking_Solver.interactive_tracking(data, orientation)

In [ ]:
Tracking_Solver.save_params()

### Generate Lineage Traces

#### Start Dask Workers

In [ ]:
dask_controller = tr.trcluster.dask_controller(
    walltime="1:00:00",
    local=False,
    n_workers=50,
    n_workers_min=50,
    memory="8GB",
    working_directory=dask_wd,
)
dask_controller.startdask()

In [ ]:
dask_controller.displaydashboard()

In [ ]:
Tracking_Solver = tr.tracking.tracking_solver(
    headpath, 
    "fluorsegmentation", 
    paramfile=True, 
    size_estimation=True,
    size_estimation_method='Perimeter/Area',
    props_list=['area'], 
    props_to_unpack={},
    pixel_scaling_factors={'area': 2},
    intensity_props_list=['mean_intensity'] 
)

In [ ]:
Tracking_Solver.compute_all_lineages(dask_controller,entries_per_partition = 100000, overwrite=False)

#### Stop Dask Workers

In [ ]:
dask_controller.shutdown()